In [307]:
%run ../config/config.py
#from config.config import *
import pandas as pd
import numpy as np
import debug
import cv2

In [308]:
image = cv2.imread(BLACK_YELLOW_UP)
image_size = (SIZE,SIZE)
image = cv2.resize(image,image_size)

In [309]:
def _detect_edges(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, THRESHOLD1, THRESHOLD2, EDGES)
    return edges

edges_image = _detect_edges(image)

In [310]:
def _generate_edges_lines(canny):
    
    lines = cv2.HoughLines(canny, 
                            rho = RHO_RESOLUTION, 
                            theta =  THETA_RESOLUTION, 
                            threshold = HOUGH_THRESHOLD)
    
    #Format dataset
    lines = lines.reshape(len(lines),2)
    
    coor_lines = []
    for line in lines:
        rho,theta = line
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        x1 = int(x0 +DISTANCE*(-b))
        y1 = int(y0 +DISTANCE*(a))
        x2 = int(x0 -DISTANCE*(-b))
        y2 = int(y0 -DISTANCE*(a))   
        coor_lines.append(((x1,y1),(x2,y2)))
    
    return np.array(coor_lines)
    
lines = _generate_edges_lines(edges_image)

#debug.lines(image, lines, color = RED, size = 4)

In [311]:
from sklearn.cluster import KMeans

def _get_lines_means(points, direction, n_clusters = 9):
    
    means = []

    for lines in points:
    
        kmeans = KMeans(n_clusters = n_clusters, n_init=10)
        kmeans.fit(lines)
        labels = kmeans.labels_
    
        unique_labels = list(range(n_clusters))
    
        cluster_means = []
    
        for label in unique_labels: 
            cluster_data = lines[labels == label] 
            cluster_mean = np.round(sum(cluster_data)/len(cluster_data))  
            cluster_means.append(cluster_mean)
        
        cluster_means = np.array(cluster_means)
        sorted_index = np.argsort(cluster_means[:, direction])
        
        means.append(cluster_means[sorted_index])
   
    #debug.scatter(cluster_means, labels = unique_labels, cmap = 'viridis')
 
    return means


In [312]:


def _get_vertical_averages(lines):
    
    vertical_lines = lines[lines[:,0,1] > SIZE]
    
    initial_points = vertical_lines[:,1][vertical_lines[:,1,1] < -SIZE]
    final_points = vertical_lines[:,0][vertical_lines[:,0,1] > SIZE] 
    
    averages = _get_lines_means((initial_points, final_points), VERTICAL)
    
    return np.stack(averages, axis=1)

def _get_horizontal_averages(lines):

    horizontal_lines = lines[lines[:,1,0] > SIZE]
    
    initial_points = horizontal_lines[:,0][horizontal_lines[:,0,0] < -SIZE]
    final_points = horizontal_lines[:,1][horizontal_lines[:,1,0] > SIZE]
    
    averages = _get_lines_means((initial_points, final_points), HORIZONTAL)
    
    return np.stack(averages, axis=1)
    
vertical_lines = _get_vertical_averages(lines).astype(int)
horizontal_lines = _get_horizontal_averages(lines).astype(int)

lines = np.concatenate((vertical_lines,horizontal_lines), axis=0)

#debug.lines(image,lines, size = 3, color=GREEN);


In [313]:
def _detect_inner_corners(lines, a = 9):

    points = []
    for i in range(a):
        for j in range(a,a*2):

            line1 = lines[i].tolist()
            line2 = lines[j].tolist()
            xdiff = (line1[0][0] - line1[1][0], line2[0][0] - line2[1][0])
            ydiff = (line1[0][1] - line1[1][1], line2[0][1] - line2[1][1])

            def det(a, b):
                return a[0] * b[1] - a[1] * b[0]

            div = det(xdiff, ydiff)
            if div == 0:
                raise Exception('lines do not intersect')

            d = (det(*line1), det(*line2))
            x = det(d, xdiff) / div
            y = det(d, ydiff) / div
            points.append((x, y))

    return points
debug.lines(image,lines, size = 1);
points =  _detect_inner_corners(lines)

points;


In [314]:
def _get_middle_squares(lines):

    middle_lines = []

    for i in range(17):
        
        if i == 8: continue
        
        mean = (lines[i] +lines[i+1])/2
        
        middle_line = mean.astype(int)
        middle_lines.append(middle_line)
        
    corners = _detect_inner_corners(middle_lines, 8)
    return corners

middle_points = _get_middle_squares(lines)
debug.points(image, middle_points, color = BLUE, size =7)
print(middle_points)


[(104.88928330457546, 75.52133405936453), (103.89218669434005, 141.97070101290893), (102.792450344517, 215.2602734689751), (101.55141316570635, 297.96653688542636), (100.31182330162333, 380.5763471132453), (99.12675118583483, 459.55293882972103), (97.99150728720292, 535.2088357885481), (96.92026926922439, 606.5991979866891), (180.4042848455792, 77.30260285911659), (179.431324937164, 143.72491465556115), (178.36487373698367, 216.52971732152903), (177.16169195786287, 298.66888316931204), (175.95041655214973, 381.36058708616815), (174.793564364649, 460.3369108133529), (173.6944584002158, 535.370998482828), (172.6558610152305, 606.2742687407284), (256.2338134538422, 79.0912908105624), (255.5695587196941, 145.4930408420079), (254.84608647760643, 217.8144267564144), (254.03011016155574, 299.38291635019664), (253.20202806906852, 382.16155123847085), (252.41195038348033, 461.1411027370921), (251.66771536881802, 535.5380243813684), (250.9634610454384, 605.9383047792115), (333.3317235365604, 80.

In [315]:
def _distance(a, b):
    
    x_diff = abs(a[0] - b[0])
    y_diff = abs(a[1] - b[1])
    
    distance = np.sqrt(x_diff**2 + y_diff**2)
    
    return distance

In [316]:
def _get_grid(middle_points, lines):
    grid_points = np.array(_detect_inner_corners(lines))
    
    grid = []
    
    for middle_point in middle_points:
        
        distances = []
        
        for point in grid_points:
            
            distances.append(_distance(middle_point, point))
            
        index = np.argsort(distances)
        
        order = grid_points[index]
        
        final = order[:4].tolist()
        
        grid.append(final)
        
    return grid

grid = _get_grid(middle_points, lines)

debug.points(image, grid[0], color = GREEN, size = 10)


array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

In [317]:

cv2.imshow("Image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()